<a href="https://colab.research.google.com/github/atharvavyas1/Finance-N8N-project/blob/main/finviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# FINVIZ LIBRARY - CORRECTED VERSION
# Working code for the finviz package
# ============================================

import finviz
import pandas as pd

# ============================================
# 1. GET STOCK DATA (Fundamentals + News)
# ============================================

def get_stock_data(ticker):
    """
    Get all available data for a stock including news
    """
    try:
        # Get stock data - returns a dictionary
        stock_data = finviz.get_stock(ticker)

        print(f"\nData for {ticker}:")
        print("="*60)

        # Display key metrics
        key_metrics = ['Price', 'Change', 'Volume', 'Market Cap', 'P/E',
                      'EPS (ttm)', '52W High', '52W Low', 'RSI (14)', 'Sector', 'Industry']

        for metric in key_metrics:
            if metric in stock_data:
                print(f"{metric}: {stock_data[metric]}")

        return stock_data

    except Exception as e:
        print(f"Error: {e}")
        return {}

# ============================================
# 2. GET NEWS FOR A STOCK
# ============================================

def get_news(ticker):
    """
    Extract news from finviz data
    """
    try:
        # Get all stock data
        stock_data = finviz.get_stock(ticker)

        # News is typically in the data but not well documented
        # We'll need to check what keys are available
        print(f"\nAvailable data keys for {ticker}:")
        print(list(stock_data.keys())[:20])  # Show first 20 keys

        return stock_data

    except Exception as e:
        print(f"Error: {e}")
        return {}

# ============================================
# 3. STOCK SCREENER
# ============================================

def screen_stocks():
    """
    Screen stocks using various filters
    """
    try:
        # Example: Get all stocks with certain criteria
        # The screener returns a list of tickers meeting criteria

        # Get oversold tech stocks
        filters = ['ta_rsi_os30', 'sec_technology']  # RSI < 30, Tech sector
        stock_list = finviz.get_screener(filters=filters)

        if stock_list:
            print(f"\nFound {len(stock_list)} stocks matching criteria")
            print("First 10 tickers:", stock_list[:10])

            # Get data for first few stocks
            results = []
            for ticker in stock_list[:5]:
                data = finviz.get_stock(ticker)
                results.append({
                    'Ticker': ticker,
                    'Price': data.get('Price', ''),
                    'RSI': data.get('RSI (14)', ''),
                    'Change': data.get('Change', '')
                })

            df = pd.DataFrame(results)
            print("\nScreener Results:")
            print(df)

            return df

    except Exception as e:
        print(f"Screener error: {e}")
        return pd.DataFrame()

# ============================================
# 4. GET INSIDER TRADING
# ============================================

def get_insider_info(ticker):
    """
    Get insider trading information
    """
    try:
        # Get insider trading data
        insider_data = finviz.get_insider(ticker)

        print(f"\nInsider Trading for {ticker}:")
        print("="*60)

        # Convert to DataFrame for better display
        df = pd.DataFrame(insider_data)
        print(df.head(10))

        return df

    except Exception as e:
        print(f"Insider trading error: {e}")
        return pd.DataFrame()

# ============================================
# 5. GET NEWS USING FINVIZ (Alternative Method)
# ============================================

def get_news_alternative(ticker):
    """
    Get news by parsing the finviz page directly
    """
    import requests
    from bs4 import BeautifulSoup

    url = f"https://finviz.com/quote.ashx?t={ticker}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the news table
        news_table = soup.find('table', {'id': 'news-table'})

        if news_table:
            news_items = []
            rows = news_table.find_all('tr')

            current_date = None
            for row in rows[:20]:  # Get first 20 news items
                cells = row.find_all('td')

                if len(cells) == 2:
                    # Date/time cell and news cell
                    time_cell = cells[0].text.strip()
                    if len(time_cell) > 8:  # This is a date
                        current_date = time_cell
                    else:  # This is just time
                        time_cell = f"{current_date} {time_cell}" if current_date else time_cell

                    news_cell = cells[1]
                    link = news_cell.find('a')

                    if link:
                        news_items.append({
                            'datetime': time_cell,
                            'title': link.text.strip(),
                            'url': link.get('href'),
                            'source': link.get('href').split('/')[2] if link.get('href') else ''
                        })

            print(f"\nNews for {ticker}:")
            print("="*60)
            for i, item in enumerate(news_items[:10], 1):
                print(f"{i}. {item['title'][:80]}...")
                print(f"   Time: {item['datetime']}")
                print()

            return pd.DataFrame(news_items)

    except Exception as e:
        print(f"News scraping error: {e}")

    return pd.DataFrame()

# ============================================
# MAIN EXECUTION
# ============================================

# Test 1: Get stock data
print("="*60)
print("STOCK DATA TEST")
print("="*60)
stock_info = get_stock_data('AAPL')

# Test 2: Get news
print("\n" + "="*60)
print("NEWS TEST")
print("="*60)
news_df = get_news_alternative('AAPL')

# Test 3: Stock screener
print("\n" + "="*60)
print("SCREENER TEST")
print("="*60)
screener_df = screen_stocks()

# Test 4: Insider trading
print("\n" + "="*60)
print("INSIDER TRADING TEST")
print("="*60)
insider_df = get_insider_info('AAPL')

# Show what functions are actually available in finviz
print("\n" + "="*60)
print("AVAILABLE FINVIZ FUNCTIONS:")
print("="*60)
print([func for func in dir(finviz) if not func.startswith('_')])